# Seseme CSM Fine-Tuning

This notebook demonstrates how to fine-tune the Seseme CSM model using the Hugging Face `transformers` library.

## Setup

Connect to the Hugging Face Hub and install the dataset. The dataset used is in `parquet` format, which is efficient for large datasets.

In [ ]:
from huggingface_hub import login

login()

In [ ]:
!git clone https://huggingface.co/datasets/MrDragonFox/Elise

Cloning into 'Elise'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17 (from 1)
Unpacking objects: 100% (17/17), 5.95 KiB | 1.98 MiB/s, done.


## Preprocessing

In [ ]:
from transformers import CsmForConditionalGeneration, AutoProcessor, Trainer, TrainingArguments
from datasets import load_dataset, Audio
import torch
import numpy as np
from tqdm import tqdm
import os

model_id = "sesame/csm-1b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
processor = AutoProcessor.from_pretrained(model_id)
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)
model.train()
model.codec_model.eval()  # Keep codec model in eval mode during training

# Path to the dataset after git clone
dataset_path = os.path.join("content", "Elise", "data", "train-00000-of-00001.parquet")

# Check if the path exists, if not, adjust for current directory structure
if not os.path.exists(dataset_path):
    # Adjust path if running in the current directory
    dataset_path = os.path.join("Elise", "data", "train-00000-of-00001.parquet")
    # If you're running this notebook elsewhere, adjust the path accordingly

# Load the dataset from parquet file - use pandas first to avoid LocalFileSystem error
try:
    import pandas as pd
    df = pd.read_parquet(dataset_path)
    # Convert pandas DataFrame to Hugging Face Dataset
    from datasets import Dataset
    ds = {"train": Dataset.from_pandas(df)}
    print(f"Dataset structure: {ds}")
except Exception as e:
    print(f"Error loading with pandas: {e}")
    # Fallback to direct loading - may cause the error
    try:
        ds = load_dataset("parquet", data_files={"train": dataset_path})
        print(f"Dataset structure: {ds}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        # Last resort: try arrow directly
        try:
            import pyarrow.parquet as pq
            table = pq.read_table(dataset_path)
            from datasets import Dataset
            ds = {"train": Dataset.from_arrow_table(table)}
            print(f"Dataset loaded with pyarrow: {ds}")
        except Exception as e:
            print(f"All loading methods failed: {e}")
            raise

# Ensure the audio is 24kHz (CSM requirement)
# Check if 'audio' column exists, otherwise look for the correct audio column
audio_column = "audio"
if audio_column in ds["train"].column_names:
    ds["train"] = ds["train"].cast_column(audio_column, Audio(sampling_rate=24000))
else:
    print(f"Warning: Column '{audio_column}' not found. Available columns: {ds['train'].column_names}")
    # Try to identify the audio column if it has a different name
    # You might need to adjust this based on your dataset structure

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset structure: {'train': Dataset({
    features: ['audio', 'text'],
    num_rows: 1195
})}


In [ ]:
def prepare_conversation_batch(batch_size=4, offset=0):
    """Prepare a batch of conversations from the dataset"""
    batch_indices = list(range(offset, min(offset + batch_size, len(ds["train"]))))
    conversation = []

    # Use a stringified integer for the speaker ID (e.g., "0")
    speaker_id = "0"  # Changed from "Elise" to "0"

    for idx in batch_indices:
        example = ds["train"][idx]

        # Extract text and audio from the dataset
        text = example.get("text", "")

        # Handle audio data correctly based on dataset structure
        if "audio" in example and isinstance(example["audio"], dict) and "array" in example["audio"]:
            audio_data = example["audio"]["array"]
        elif "audio" in example:
            # If audio is directly accessible
            audio_data = example["audio"]
        else:
            # Try to find audio under a different key
            print(f"Warning: Audio not found in example. Available keys: {example.keys()}")
            continue

        conversation.append({
            "role": speaker_id,  # Now using a stringified integer
            "content": [
                {"type": "text", "text": text},
                {"type": "audio", "path": audio_data}
            ],
        })

    return conversation

## Modeling

In [ ]:
# Training configurations
output_dir = "./csm_elise_model"
num_train_epochs = 3
per_device_train_batch_size = 1  # CSM models can be memory intensive
gradient_accumulation_steps = 4
learning_rate = 5e-5
warmup_steps = 100
logging_steps = 10
save_steps = 500
max_steps = 2000  # Adjust based on dataset size

# Initialize training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    logging_steps=logging_steps,
    save_steps=save_steps,
    max_steps=max_steps,
    fp16=True,  # Use mixed precision training
    remove_unused_columns=False,
    report_to="tensorboard",
)

# Custom training loop (as alternative to using Trainer)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,
                                             start_factor=1.0,
                                             end_factor=0.0,
                                             total_iters=max_steps)

# Training loop
model.train()
total_batches = min(max_steps, len(ds["train"]) // per_device_train_batch_size)

for step in tqdm(range(total_batches)):
    batch_offset = step * per_device_train_batch_size
    conversation = prepare_conversation_batch(per_device_train_batch_size, batch_offset)

    # Skip empty conversations
    if not conversation:
        continue

    # Process the conversation batch
    try:
        inputs = processor.apply_chat_template(
            conversation,
            tokenize=True,
            return_dict=True,
            output_labels=True,
        ).to(device)

        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Optimizer step with gradient accumulation
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        # Log progress
        if step % logging_steps == 0:
            print(f"Step {step}, Loss: {loss.item()}")

        # Save checkpoint
        if step % save_steps == 0 and step > 0:
            model.save_pretrained(f"{output_dir}/checkpoint-{step}")
            processor.save_pretrained(f"{output_dir}/checkpoint-{step}")

    except Exception as e:
        print(f"Error in batch at offset {batch_offset}: {e}")
        continue

  0%|          | 1/1195 [00:02<48:09,  2.42s/it]

Step 0, Loss: 5.28347635269165


  1%|          | 11/1195 [00:05<06:29,  3.04it/s]

Step 10, Loss: 6.679079055786133


  2%|▏         | 21/1195 [00:08<06:23,  3.06it/s]

...

 99%|█████████▉| 1181/1195 [06:57<00:04,  3.19it/s]

Step 1180, Loss: 4.728840351104736


100%|█████████▉| 1191/1195 [07:01<00:01,  3.16it/s]

Step 1190, Loss: 5.619165897369385


100%|██████████| 1195/1195 [07:02<00:00,  2.83it/s]


Now save the trained model in local directory.

In [ ]:
# Save the final model
model.save_pretrained(f"{output_dir}/final")
processor.save_pretrained(f"{output_dir}/final")

[]

## Inference

Try the model with some sample inputs to see how it performs.

In [ ]:
from IPython.display import Audio, display
import soundfile as sf

# another equivalent way to prepare the inputs
conversation = [
    {"role": "0", "content": [{"type": "text", "text": "The past is just a story we tell ourselves."}]},
]
inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Move tensor from GPU to CPU before converting to numpy
audio_cpu = audio[0].to(torch.float32).cpu().numpy()  # This fixes the TypeError

# Write to file and play
output_file = "output.wav"
sf.write(output_file, audio_cpu, 24000)
display(Audio(output_file))

In [ ]:
# another equivalent way to prepare the inputs
conversation = [
    {"role": "0", "content": [{"type": "text", "text": "That's so sweet <moans> I know <breathes deeply>. It feels so good."}]},
]
inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Move tensor from GPU to CPU before converting to numpy
audio_cpu = audio[0].to(torch.float32).cpu().numpy()  # This fixes the TypeError

# Write to file and play
output_file = "output_2.wav"
sf.write(output_file, audio_cpu, 24000)
display(Audio(output_file))

## Save To Hugging Face Hub

Finally, save the model to the Hugging Face Hub for future use or sharing.

In [ ]:
# Install the huggingface_hub package if not already installed
# !pip install huggingface_hub

from huggingface_hub import HfApi

# Define your model repository name (create this on the HF website first)
model_name = "keanteng/sesame-csm-elise"  # Replace with your desired repo name

# Alternatively, push an already saved model directory
api = HfApi()
api.upload_folder(
    folder_path=f"{output_dir}/final",
    repo_id=model_name,
    repo_type="model"
)

model-00002-of-00002.safetensors:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
# Add a model card with description
with open(f"{output_dir}/README.md", "w") as f:
    f.write("""
# CSM Elise Voice Model

This model is a fine-tuned version of [sesame/csm-1b](https://huggingface.co/sesame/csm-1b) using the [Elise dataset](https://huggingface.co/datasets/MrDragonFox/Elise).

## Model Details
- **Base Model**: sesame/csm-1b
- **Training Data**: MrDragonFox/Elise dataset
- **Fine-tuning Approach**: Voice cloning through conditional speech generation
- **Voice Characteristics**: [Describe voice qualities]
- **Training Parameters**:
  - Learning Rate: 5e-5
  - Epochs: 3
  - Batch Size: 1 with gradient accumulation steps of 4

""")

In [ ]:
api.upload_file(
    path_or_fileobj=f"{output_dir}/README.md",
    path_in_repo="README.md",
    repo_id=model_name,
    repo_type="model"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/hf_api.py:9635: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


CommitInfo(commit_url='https://huggingface.co/keanteng/sesame-csm-elise/commit/e41344358829eccf0fa8c2d2c8577f17625a81f1', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='e41344358829eccf0fa8c2d2c8577f17625a81f1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/keanteng/sesame-csm-elise', endpoint='https://huggingface.co', repo_type='model', repo_id='keanteng/sesame-csm-elise'), pr_revision=None, pr_num=None)